In [1]:
from keras.models import Model, Sequential
from keras.layers import MaxPooling2D, Conv2D, Flatten, Activation, Dense
from PIL import Image as Im
from IPython.display import Image
import numpy as np
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

/home/letfoolsdie/virt_envs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [19]:
train_data = unpickle("../data/processed_data/all_cf_batches.p")

with open("../data/processed_data/all_labels.txt") as lab:
    train_labels = [int(i) for i in lab.read().split()]
    
label_names = {
    0:'airplane',
    1:'automobile',
    2:'bird',
    3:'cat',
    4:'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
}

In [20]:
def to_categorical(data, num_classes):
    temp = np.zeros((len(data), num_classes))
    temp[range(len(data)), data] = 1
    return temp

train_labels = to_categorical(train_labels, 10)

In [21]:
train_labels[:10]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

In [26]:
train_data = train_data / 255

In [27]:
train_data, X_test, train_labels, Y_test = train_data[:45000], train_data[45000:], train_labels[:45000], train_labels[45000:]

In [28]:
X_train, X_val, Y_train, Y_val = train_data[:40000], train_data[40000:], train_labels[:40000], train_labels[40000:]

In [29]:
print("Train data:")
print(X_train.shape)
print("="*20)
print("Validation data:")
print(X_val.shape)
print("="*20)
print("Test data:")
print(X_test.shape)

Train data:
(40000, 32, 32, 3)
Validation data:
(5000, 32, 32, 3)
Test data:
(5000, 32, 32, 3)


In [30]:
# ind = 12411

# print(label_names[Y_train[ind]])
# plt.imshow(X_train[ind]);

In [31]:
def build_model():
    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding='same',
                     input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    return model

model = build_model()              

In [33]:
batch_size = 32
epochs = 5

model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_val, Y_val))

Train on 40000 samples, validate on 5000 samples
Epoch 1/5
40000/40000 [==============================] - 92s - loss: 0.8430 - acc: 0.7076 - val_loss: 0.9539 - val_acc: 0.6740
Epoch 2/5
40000/40000 [==============================] - 93s - loss: 0.8030 - acc: 0.7211 - val_loss: 0.9321 - val_acc: 0.6760
Epoch 3/5
40000/40000 [==============================] - 96s - loss: 0.7658 - acc: 0.7339 - val_loss: 0.9154 - val_acc: 0.6832
Epoch 4/5
40000/40000 [==============================] - 98s - loss: 0.7389 - acc: 0.7436 - val_loss: 0.9227 - val_acc: 0.6868
Epoch 5/5
40000/40000 [==============================] - 97s - loss: 0.7077 - acc: 0.7506 - val_loss: 0.8702 - val_acc: 0.7060


In [36]:
from keras import metrics
preds = model.predict(X_test)

In [38]:
# metrics.categorical_accuracy(Y_test, preds)
model.evaluate(X_test, Y_test)

4960/5000 [============================>.] - ETA: 0s

[0.8343838067054749, 0.7154]

In [39]:
model.save_weights("keras_cifar10_default_model_10ep_70accur.h5")

In [40]:
from keras.models import model_from_json
json_string = model.to_json()

In [42]:
with open('default_model_keras.json', 'w') as mod:
    mod.write(json_string)